In [137]:
import numpy as np
import pandas as pd
import requests

In [138]:
def fetch_data_api(api, city, nodays):
    """
    Get weather data from weather API. Returns empty string if data not found'

    """
    #api = '7aa7fc7be1af4500882144440212208'
    #city = 'Paris'
    #nodays = '7'
    url = f'http://api.weatherapi.com/v1/forecast.json?key={api}&q={city}&days={nodays}&aqi=no&alerts=no'

    response = requests.get(url)

    if response.status_code != 200:
        return ''
    data = response.json()

    data = data['forecast']['forecastday']
    return data

In [140]:
def jour_annee(x):
    return x.toordinal() - dt.date(x.year, 1, 1).toordinal() + 1

In [147]:
data = fetch_data_api('09e033659f124258acf74721212408','Paris','3')
daily_data = pd.DataFrame(data[0]['day']).iloc[2,:]
### gives the weather hour by hour for the entire first day###

daily_hourly_data = pd.DataFrame(data[0]['hour'])
#condition has a code and an image, the code below only keeps the needed weather state
for i in range(len(daily_hourly_data)):
    interm = daily_hourly_data['condition'][i]['text']
    daily_hourly_data['condition'][i]=interm

#condition is given as three values this keeps only one
data_series=[]
for i in range(3):
    data_series.append(data[i]['hour'])

#condition has a code and an image, the code below only keeps the needed weather state
for x in range(3):
    for y in range(24):
        data_series[x][y]['condition']=data_series[x][y]['condition']['text']

d={}
for i in range(len(data_series)):
    d[i] = pd.DataFrame(data_series[i])

seven_day_forecast = pd.concat([d[i] for i in range(len(data_series))])

#convert time column to datetime
seven_day_forecast['time']=pd.to_datetime(seven_day_forecast['time'])

#change conditions for a more simple range of possible values
clear = ['Clear', 'Sunny']
clouds = ['Clouds','Partly cloudy', 'Cloudy', 'Overcast']
rain = ['Rain','Patchy rain possible', 'Patchy light rain', 'Light rain', 'Moderate rain at times','Moderate rain','Heavy rain at times','Heavy rain','Moderate or heavy rain shower','Torrential rain shower']
mist = ['Mist']
drizzle = ['Drizzle','Patchy light drizzle','Light drizzle','Freezing drizzle','Heavy freezing drizzle','Light rain shower']
fog = ['Fog', 'Freezing Fog']
snow = ['Snow','Patchy snow possible', 'Patchy sleet possible', 'Patchy freezing drizzle possible', 'Blowing snow','Blizzard','Moderate or heavy freezing rain','Light freezing rain','Light sleet', 'Moderate or heavy sleet','Patchy light snow','Light snow','Patchy moderate snow','Moderate snow','Patchy heavy snow','Heavy snow','Ice pellets','Light sleet showers', 'Moderate or heavy sleet showers','Light snow showers','Moderate or heavy snow showers','Light showers of ice pellets','Moderate or heavy showers of ice pellets']
thunderstorm = ['Thunderstorm','Thundery outbreaks possible', 'Patchy light rain with thunder','Moderate or heavy rain with thunder','Patchy light snow with thunder','Moderate or heavy snow with thunder']
haze = ['Haze']

summed_conditions = [clear, clouds, rain, mist, drizzle, fog, snow, thunderstorm, haze]
seven_day_forecast=seven_day_forecast.reset_index()
for i in range(len(seven_day_forecast['condition'])):
    for x in summed_conditions:
        if seven_day_forecast['condition'][i] in x:
            seven_day_forecast['condition'][i] = x[0]


lunch = seven_day_forecast[seven_day_forecast['time'].dt.hour==12]
lunch['service']='Lunch'
dinner = seven_day_forecast[seven_day_forecast['time'].dt.hour==18]
dinner['service']='Dinner'

forecasted_services = pd.concat([lunch, dinner])
forecasted_services['time'] = forecasted_services['time'].dt.date

forecasted_services=forecasted_services.set_index('time')
forecasted_services = forecasted_services.sort_values('time')
forecasted_services = forecasted_services[['index','temp_c','condition','wind_kph','feelslike_c', 'service']]
forecasted_services=forecasted_services.reset_index().rename(columns={'time':'date'}).drop(columns='index')
print(forecasted_services)
forecasted_services=forecasted_services.rename(columns={'feelslike_c':'feels_like','temp_c':'temp', 'wind_kph':'wind_speed'}, errors = 'raise')
forecasted_services['date'] = pd.to_datetime(forecasted_services['date'],  format='%Y-%m-%d')
forecasted_services['jour_de_sem'] = forecasted_services['date'].dt.weekday +1
forecasted_services['jour_du_mois'] = forecasted_services['date'].dt.day
forecasted_services['mois_de_annee'] = forecasted_services['date'].dt.month
forecasted_services['sem_de_annee'] = forecasted_services['date'].dt.week
forecasted_services['jour_annee'] = forecasted_services.apply(lambda x:jour_annee(x['date']), axis=1)

forecasted_services['temp_min'] = forecasted_services['temp'] - 1
forecasted_services['temp_max'] = forecasted_services['temp'] + 1
forecasted_services['clouds_all'] = 0
forecasted_services['moyen_7_services'] = 0
forecasted_services['moyen_31_services'] = 0
forecasted_services['moyenne_3der_j&service'] = 0
forecasted_services['condition'] = forecasted_services['condition'].str.lower()
table = pd.get_dummies(forecasted_services['condition'])
forecasted_services = pd.merge(forecasted_services, table, left_index=True, right_index=True).drop(columns='condition')
forecasted_services['jour']= forecasted_services['date'].dt.strftime('%A')
for i in range(len(forecasted_services)):
    #print(i)
    if forecasted_services['clouds'][i]==1:
        print(forecasted_services['clouds_all'][i])
        forecasted_services['clouds_all'][i] = 50

         date  temp_c condition  wind_kph  feelslike_c service
0  2021-08-31    20.6    Clouds      15.8         20.6   Lunch
1  2021-08-31    22.7    Clouds      17.6         24.2  Dinner
2  2021-09-01    19.6     Clear      15.8         19.6   Lunch
3  2021-09-01    20.7     Clear      18.0         20.7  Dinner
4  2021-09-02    22.9     Clear      18.7         24.1   Lunch
5  2021-09-02    25.0     Clear      18.0         25.0  Dinner


/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_90079/1095417621.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_hourly_data['condition'][i]=interm
/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_90079/1095417621.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seven_day_forecast['condition'][i] = x[0]
/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_90079/1095417621.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0
0


/var/folders/p2/6ddwj2812811pbqjm8t6brv80000gn/T/ipykernel_90079/1095417621.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecasted_services['clouds_all'][i] = 50


In [148]:
forecasted_services

,date,temp,wind_speed,feels_like,service,jour_de_sem,jour_du_mois,mois_de_annee,sem_de_annee,jour_annee,temp_min,temp_max,clouds_all,moyen_7_services,moyen_31_services,moyenne_3der_j&service,clear,clouds,jour
0,2021-08-31,20.6,15.8,20.6,Lunch,2,31,8,35,243,19.6,21.6,50,0,0,0,0,1,Tuesday
1,2021-08-31,22.7,17.6,24.2,Dinner,2,31,8,35,243,21.7,23.7,50,0,0,0,0,1,Tuesday
2,2021-09-01,19.6,15.8,19.6,Lunch,3,1,9,35,244,18.6,20.6,0,0,0,0,1,0,Wednesday
3,2021-09-01,20.7,18.0,20.7,Dinner,3,1,9,35,244,19.7,21.7,0,0,0,0,1,0,Wednesday
4,2021-09-02,22.9,18.7,24.1,Lunch,4,2,9,35,245,21.9,23.9,0,0,0,0,1,0,Thursday
5,2021-09-02,25.0,18.0,25.0,Dinner,4,2,9,35,245,24.0,26.0,0,0,0,0,1,0,Thursday
